In [1]:
import numpy as np
import itertools

## Compressed sensing

http://statweb.stanford.edu/~candes/papers/StableRecovery.pdf

$$
\begin{align}
&\text{Assume $\forall x_i \in X$;} \tag{the true signals}\\
&\quad\text{(1) $x_i \in \mathbb R^m$} \\
&\quad\text{(2) $x_i$ is sufficiently sparse} \\
&\text{Given $f(x_i) = Ax_i+e$ such that;} \tag{the function we know}\\
&\quad\text{(2) $A$ is approx orthonormal}\\ 
&\quad\text{(3) and $e$ is bounded} \\
&\text{Given $y_i = f(x_i)$ such that;} \tag{the observations}\\
&\quad\text{(2) where $n << m$} \\
x_i^* &= \text{min} \parallel \tilde x_i\parallel_1 \text{subject to} \parallel f(\tilde x_i) - y_i \parallel \le \epsilon \tag{?}\\
\end{align}
$$

Requirements.
- $x_i$ is sufficiently sparse
- $A$ is approximatley orthonormal
- $e$ is bounded (this was just needed to make the proofs easier?!? otherwise would need measures?)
- iid samples!?

#### Misunderstanding.

I thought we were matching the signals in a sparse basis.
We take our signal, $x_i$ and project it into a new basis where the signal has a sparse representation. We then match signals in this basis.

Rather the above requires $x_i$ itself to be sparse.

Oh, if $A$ is approximately orthonormal then $y_i$ will also be sparse!? No.

#### Search

Want a simple 2D case. Show how it is underdetermined.
> convex program searching, among all signals consistent with the data y


#### Structured spaces and regularisers and priors

The trick is using structure in the spaces, X/Y, to help us ...?
What about other structure in X or Y? How can that be translated into a regulariser?

Want to learn the distinct strutures of a space (e.g. smooth, sparse, ...) and then apply them when solving for x. 

## Sparse representations

Ok, so can we just learn a sparse representation?
Take an AE, use a large hidden space, and regularise it to be sparse!?



# Isometry

Let $X$ and $Y$ be metric spaces with metrics $d_X$ and $d_Y$. A map $f : X \to Y$ is called an isometry or distance preserving if for any $a,b \in X$ one has

$$d_Y( f(a), f(b)) = d_X (a, b) $$


## Restricted isometry

> This property essentially requires that every set of columns with cardinality less than
$S$ approximately behaves like an orthonormal system. [source](http://statweb.stanford.edu/~candes/papers/StableRecovery.pdf)

$$
\begin{align}
(1-\delta_S)\parallel x \parallel_2^2 \le \parallel A_Sx \parallel \le (1+\delta_S)\parallel x \parallel_2^2 \\
\end{align}
$$

I am confused. If $A_{S_N}$ (all the columns) satisfies RIP then shouldnt the other $A_S$?

$$
\begin{align}
\parallel A_s^*A_s-I\parallel_{2\to 2}\leq \delta_s \\
\end{align}
$$


> A challenging aspect of RIP is its computational cost. Indeed, RIP is a property of the submatrices of a specific size. At present, no subexponential-time algorithm is known for testing RIP. [source](https://arxiv.org/pdf/0812.3137.pdf)



In [2]:
def combinations(n, S):
    if S == 1:
        return list(itertools.combinations(range(n), 1))
    else:
        return list(itertools.combinations(range(n), S)) + combinations(n, S-1)

In [3]:
def approx_orth(A, t):
    A_s = A[:, t]
    u, s, v = np.linalg.svd(np.dot(A_s.conj().T, A_s))
    e_S = np.abs(s - np.ones(len(t)))
    return np.max(e_S)

In [4]:
def ft_matrix(N):
    i, j = np.meshgrid(np.arange(N), np.arange(N))
    omega = np.exp( - 2 * np.pi * 1J / N )
    W = np.power( omega, i * j ) / np.sqrt(N)
    return W

In [5]:
def rim(A, S):
    T = combinations(A.shape[-1], S)
    return np.max([approx_orth(A, t) for t in T])

In [11]:
[rim(ft_matrix(15), i) for i in range(1, 15)]

[1.0658141036401503e-14,
 1.099120794378905e-14,
 1.2545520178264269e-14,
 1.3433698597964394e-14,
 1.3766765505351941e-14,
 1.4210854715202004e-14,
 1.4432899320127035e-14,
 1.4432899320127035e-14,
 1.4432899320127035e-14,
 1.4432899320127035e-14,
 1.4432899320127035e-14,
 1.4432899320127035e-14,
 1.4432899320127035e-14,
 1.4432899320127035e-14]

In [42]:
A = np.random.standard_normal((10, 10))
A /= np.linalg.norm(A, axis=1, keepdims=True)
# axis = 0 or 1!?

[rim(A, i) for i in range(1, A.shape[-1])]

[0.7082281239704467,
 1.1847212362098167,
 1.41341056257362,
 1.5968500301894166,
 1.6782821675460822,
 1.7341228419466281,
 1.7639518661661806,
 1.7694464508204697,
 1.7699056758830496]

## Non-linear RIP

Relationship to the gradient!? (and hessian?! where was that reference!?!?)

## Modelling the data.

Can use signals in a sparse basis.
Thus the model is many causes/bases -> image.

What about other types of model?


